In [2]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((301619, 62), (148560, 62))

In [29]:
#train.describe()
train = train_.sample(10000)
validate = validate_.sample(3000)
train_.shape, #validate_.shape, validate.head(2)

((301619, 62),)

In [30]:
train = train.query('SUB2 != 1')
validate = validate.query('SUB2 != 1')
print train['SUB2'].value_counts()

4     1707
2     1409
3      768
7      379
20     376
10     341
13     204
5      143
11      60
9       38
16      35
8       24
12      15
18      10
6       10
17       9
14       7
15       7
Name: SUB2, dtype: int64


In [31]:
# for sub2



X_train = train.drop(['SUB2'], axis=1)
Y_train = train["SUB2"]

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:    
    if 'FLG' in n: 
        pass
    else:
        print 'F-score: %3.2ft for feature %s' % (s,n)
                
#print X_train.columns.tolist()

F-score: 1.43t for feature ARRESTS
F-score: 1.63t for feature ETHNIC
F-score: 2.17t for feature PREG
F-score: 2.83t for feature PRIMINC
F-score: 2.88t for feature RACE
F-score: 3.07t for feature DETNLF
F-score: 3.24t for feature DAYWAIT
F-score: 3.29t for feature PRIMPAY
F-score: 3.30t for feature VET
F-score: 4.75t for feature PSYPROB
F-score: 5.02t for feature LIVARAG
F-score: 5.10t for feature EMPLOY
F-score: 5.62t for feature FRSTUSE1
F-score: 5.70t for feature SUB3
F-score: 6.15t for feature ROUTE3
F-score: 6.43t for feature CBSA10
F-score: 6.71t for feature GENDER
F-score: 6.83t for feature METHUSE
F-score: 6.86t for feature HLTHINS
F-score: 7.02t for feature MARSTAT
F-score: 8.66t for feature EDUC
F-score: 10.03t for feature DETCRIM
F-score: 10.88t for feature FREQ1
F-score: 11.08t for feature PSOURCE
F-score: 11.21t for feature FRSTUSE3
F-score: 11.82t for feature ROUTE1
F-score: 12.33t for feature FREQ3
F-score: 13.64t for feature STFIPS
F-score: 13.83t for feature NUMSUBS
F-s

In [32]:
# get the sorted feature list
import sys
for n, s in ans:
    
    if 'FLG' in n: 
        pass
    else:
        sys.stdout.write('\'%s\',' % (n))


'ARRESTS','ETHNIC','PREG','PRIMINC','RACE','DETNLF','DAYWAIT','PRIMPAY','VET','PSYPROB','LIVARAG','EMPLOY','FRSTUSE1','SUB3','ROUTE3','CBSA10','GENDER','METHUSE','HLTHINS','MARSTAT','EDUC','DETCRIM','FREQ1','PSOURCE','FRSTUSE3','ROUTE1','FREQ3','STFIPS','NUMSUBS','SUB1','YEAR','CASEID','NOPRIOR','REGION','DIVISION','SERVSETA','AGE','DSMCRIT','IDU','FREQ2','ROUTE2','ALCDRUG','FRSTUSE2',

In [33]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'
             ]
retain_list = ['EMPLOY','GENDER','FREQ1','YEAR','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'REGION','NOPRIOR','DIVISION','DSMCRIT','ROUTE1','SUB1','AGE','IDU','SUB3','ROUTE3',
               'FREQ3','FRSTUSE3','FREQ2','FRSTUSE2']
X_train = train[retain_list]
Y_train = train["SUB2"]
X_validate = validate[retain_list]
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((5542, 23), (1696, 23))

In [34]:
print X_train.columns.tolist()

['EMPLOY', 'GENDER', 'FREQ1', 'YEAR', 'EDUC', 'PSYPROB', 'PSOURCE', 'SERVSETA', 'DETCRIM', 'REGION', 'NOPRIOR', 'DIVISION', 'DSMCRIT', 'ROUTE1', 'SUB1', 'AGE', 'IDU', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ2', 'FRSTUSE2']


In [35]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((5542, 184), (1696, 184))

In [36]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


61.73


In [37]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


55.9


In [38]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


53.77


In [39]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

59.02


In [41]:
from sklearn.ensemble import GradientBoostingClassifier,\
 AdaBoostClassifier, RandomForestClassifier
#Adaboost 
ada_boost = AdaBoostClassifier(random_state=1851)
ada_boost.fit(X_train, Y_train) 

acc_ada = round(ada_boost.score(X_validate, Y_validate) * 100, 2)
print acc_ada


42.39


In [42]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


54.13


In [43]:
# Linear SVC
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_validate, Y_validate) * 100, 2)
print acc_linear_svc


61.73


In [44]:
print 'predict-sub2-woflags-newsplit-sample20000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'AdaBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_ada]
    })
print models.sort_values(by='Validate Set Score', ascending=False)


predict-sub2-woflags-newsplit-sample20000
                        Model  Validate Set Score
0         Logistic Regression               61.73
4                  Linear SVC               61.73
1               Random Forest               59.02
2                  Perceptron               55.90
3  Stochastic Gradient Decent               54.13
5                    AdaBoost               42.39
